## Data preprocessing

This project uses data provided by
https://touringplans.com/walt-disney-world/crowd-calendar#DataSets

Thank you touringplans.com!!

### import libraries

In [188]:
import numpy as np
import pandas as pd
#import time
import datetime
#import matplotlib.pyplot as plt
#import seaborn as sns
#sns.set()
import os
import dill
import requests
from itertools import combinations
import googlemaps

### create a list of ride names

In [189]:
# each ride has a csv file of historical wait time data
# load a sample to inspect

df = pd.read_csv(path+'enchanted_tiki_rm.csv')
df[df['SPOSTMIN']>0].shape

(14398, 4)

In [190]:
# some ride csv files are empty or have too few data points
# -999 is the code for ride closure, so valid data points are > 0
# create a list of ride names with valid csv files

path = "/Users/melhaley/Documents/TheDataIncubator/Capstone/ParkHacker/data/touring-plan/"
ride_files = [file for file in os.listdir(path) if file.endswith('.csv')]
ride_names = []
for ride in ride_files:
    df = pd.read_csv(path+ride)
    if df[df['SPOSTMIN']>0].shape[0] > 50000:
        ride_names.append(os.path.splitext(ride)[0])

In [191]:
len(ride_names)

36

### preprocess metadata

In [192]:
# the touring-plan dataset includes a metadata file of 190 variables, we will use a subset.

raw_metadata = pd.read_csv('/Users/melhaley/Documents/TheDataIncubator/Capstone/ParkHacker/data/metadata.csv')
raw_metadata.head()

,DATE,WDW_TICKET_SEASON,DAYOFWEEK,DAYOFYEAR,WEEKOFYEAR,MONTHOFYEAR,YEAR,SEASON,HOLIDAYPX,HOLIDAYM,...,HSFIREWKS,AKPRDDAY,AKPRDDT1,AKPRDDT2,AKPRDDN,AKFIREN,AKSHWNGT,AKSHWNT1,AKSHWNT2,AKSHWNN
0,01/01/2015,NaN,5,0,0,1,2015,CHRISTMAS PEAK,0,5,...,1,0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN
1,01/02/2015,NaN,6,1,0,1,2015,CHRISTMAS,2,5,...,1,0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN
2,01/03/2015,NaN,7,2,0,1,2015,CHRISTMAS,3,0,...,1,0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN
3,01/04/2015,NaN,1,3,1,1,2015,CHRISTMAS,4,0,...,1,0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN
4,01/05/2015,NaN,2,4,1,1,2015,CHRISTMAS,5,0,...,1,0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN


In [193]:
# select seasonal and weather feature variables in metadata file, deal with missing values

meta_columns = ['DATE','DAYOFWEEK', 'DAYOFYEAR', 'WEEKOFYEAR', 'MONTHOFYEAR', 'YEAR', 'SEASON',
              'HOLIDAYN', 'HOLIDAY', 'WEATHER_WDWPRECIP', 'WDWMINTEMP', 'WDWMEANTEMP', 'WDWMAXTEMP']

metadata = raw_metadata[meta_columns].copy()
metadata.columns = metadata.columns.str.lower()
metadata['holidayn'] = metadata['holidayn'].fillna(value='no_holiday')
metadata['season'] = metadata['season'].fillna(value='off_season')
metadata = metadata.dropna(axis=0) # this is approx. 6 rows

with open('metadata', 'wb') as f:
    dill.dump(metadata, f)

### generate feature matrix and labels for each ride

In [196]:
def get_ride_Xy(ride_names, path, clean_metadata):
    '''Creates dict of X (feature matrix) and y (labels) for each ride'''
    ride_features = {}
    ride_targets = {}
    for ride in ride_names:
        location = path+ride+".csv"
        df = pd.read_csv(location)
# drop -999 (closed code) and outliers
        df = df.where(df['SPOSTMIN'] >= 0)
        df = df.where(df['SPOSTMIN'] <300)
# use actual recorded wait, fill na with posted wait, then drop remaining na
        df['wait'] = df['SACTMIN'].mask(df['SACTMIN'].isna(), df['SPOSTMIN'])
        df = df.dropna(subset='wait', axis=0)
# convert date_string to datetime and extract epoch and hourofday
        df['datetime'] = df['datetime'].apply(get_datetime)
        df['hourofday'] = df['datetime'].apply(get_hour)
        df['epoch'] = df['datetime'].apply(get_epoch)
        ride_df = df.groupby(df['datetime']).max() # group by the longest wait in a given hour
        ride_df['bydate'] = pd.to_datetime(ride_df['date'])
# merge with  metadata
        ride_df = pd.merge(ride_df, clean_metadata, on='date')
# drop targets from features
        df_X = ride_df.drop(['SPOSTMIN', 'SACTMIN', 'wait'], axis=1)
# add to dicts
        df_X = df_X.reset_index()
        ride_features[ride] = df_X
        ride_targets[ride] = ride_df['wait']
    return ride_features, ride_targets



def get_datetime(x):
    '''returns date and hour in datetime format'''
    dt =  datetime.datetime.strptime(x,'%Y-%m-%d %H:%M:%S')
    return dt.replace(minute=0, second=0)

def get_hour(x):
    '''returns the hour'''
    return x.hour

def get_epoch(x):
    '''returns the epoch'''
    return x.timestamp()

In [198]:
ride_dict_Xy = get_ride_Xy(ride_names, path, metadata)

with open('ride-dict-Xy', 'wb') as f:
    dill.dump(ride_dict_Xy, f)

### create look-up tables for app:

#### ride dict with ride name, ride filename, ride duration

In [104]:
# data set includes entities.csv with ride names, duration, park code, etc.

entity_df = pd.read_csv('/Users/melhaley/Documents/TheDataIncubator/Capstone/ParkHacker/data/entities.csv')
entity_df.head()

,code,name,short_name,land,opened_on,duration,average_wait_per_hundred
0,AK01,It's Tough to Be a Bug!,It's Tough to Be a Bug,Discovery Island,04/22/1998,8.0,12.0
1,AK07,Kilimanjaro Safaris,Kilimanjaro Safaris,Africa,04/22/1998,20.0,4.0
2,AK11,Expedition Everest - Legend of the Forbidden M...,Expedition Everest,Asia,04/07/2006,4.0,4.0
3,AK14,Kali River Rapids,Kali River Rapids,Asia,02/15/1999,10.0,5.0
4,AK18,DINOSAUR,DINOSAUR,DinoLand U.S.A.,04/22/1998,3.5,3.0


In [105]:
# get name of ride, duration, and park for the rides in our dataset
#first we drop parks outside of the dataset
park_columns = ['code', 'short_name', 'duration']
all_parks = entity_df[park_columns].copy()
all_parks['code'] = all_parks['code'].replace(regex={r'^AK.+': 'Animal Kingdom',
                                         r'^EP.+':'Epcot Center',
                                         r'^HS.+': 'Hollywood Studios',
                                         r'^MK.+': "Magic Kingdom"})
park_list = ['Animal Kingdom', 'Epcot Center', 'Hollywood Studios', 'Magic Kingdom']
all_parks = all_parks[all_parks['code'].isin(park_list)]
all_parks = all_parks.reset_index(drop=True)
all_parks.head()

,code,short_name,duration
0,Animal Kingdom,It's Tough to Be a Bug,8.0
1,Animal Kingdom,Kilimanjaro Safaris,20.0
2,Animal Kingdom,Expedition Everest,4.0
3,Animal Kingdom,Kali River Rapids,10.0
4,Animal Kingdom,DINOSAUR,3.5


In [108]:
# our data does not include all rides for each park, so we will need to drop rows in parks_df
print(all_parks['short_name'].shape[0], len(ride_names))
all_parks['short_name']

56 36


0        It's Tough to Be a Bug
1           Kilimanjaro Safaris
2            Expedition Everest
3             Kali River Rapids
4                      DINOSAUR
5               TriceraTop Spin
6                   Na'vi River
7             Flight of Passage
8               Spaceship Earth
9                Seas with Nemo
10                  Turtle Talk
11               Living w/ Land
12                      Soarin'
13             Jrny Imagination
14                   Test Track
15            Frozen Ever After
16            Msn: SPACE Orange
17             Remy's Adventure
18             Gran Fiesta Tour
19             Muppet*Vision 3D
20                   Slinky Dog
21                Alien Saucers
22              Runaway Railway
23            Millennium Falcon
24           Rise of Resistance
25               Racing Academy
26                 Rock Coaster
27                   Star Tours
28             Toy Story Mania!
29              Tower of Terror
30       Meet Mickey and Minnie
31      

In [109]:
# remove the rides without valid csv data files

parks_df = all_parks.drop([0,3,5,9,10,11,13,14,15,16,17,18,19,22,23,24,25,27,29,30])
parks_df.shape

(36, 3)

In [153]:
# add ride_names to parks df and save to csv

parks_df = parks_df.sort_values(by=['short_name'], key=lambda col: col.str.lower())
parks_df['ride'] = sorted(ride_names)
parks_df.sort_index().to_csv('parks_df.csv')

In [161]:
parks_df.head()

,code,short_name,duration,ride
38,Magic Kingdom,7 Dwarfs Train,3.0,7_dwarfs_train
21,Hollywood Studios,Alien Saucers,2.5,alien_saucers
51,Magic Kingdom,Astro Orbiter,1.5,astro_orbiter
50,Magic Kingdom,Barnstormer,2.0,barnstormer
33,Magic Kingdom,Big Thunder Mtn,7.0,big_thunder_mtn


In [184]:
parks_df['duration'].loc[(parks_df['short_name'] == 'Spaceship Earth')].item()

float

### Get ride coordinates from api.themeparks.wiki

In [15]:
# api call to get ids for Walt Disney World entities
rides_get = requests.get('https://api.themeparks.wiki/v1/entity/e957da41-3552-4cf6-b636-5babc5cbc4e5/children').json()

In [16]:
# create data frame and inspect
rides_get_df = pd.DataFrame(rides_get)
rides_get_df.head()

,id,name,entityType,timezone,children
0,e957da41-3552-4cf6-b636-5babc5cbc4e5,Walt Disney World® Resort,DESTINATION,America/New_York,"{'id': '8b5b9a00-b2cc-4bd9-bf39-9c312c17e8cb',..."
1,e957da41-3552-4cf6-b636-5babc5cbc4e5,Walt Disney World® Resort,DESTINATION,America/New_York,"{'id': 'd58db4d0-3bad-4655-937b-1cbc9ed0e880',..."
2,e957da41-3552-4cf6-b636-5babc5cbc4e5,Walt Disney World® Resort,DESTINATION,America/New_York,"{'id': '75ea578a-adc8-4116-a54d-dccb60765ef9',..."
3,e957da41-3552-4cf6-b636-5babc5cbc4e5,Walt Disney World® Resort,DESTINATION,America/New_York,"{'id': '47f90d2c-e191-4239-a466-5892ef59a88b',..."
4,e957da41-3552-4cf6-b636-5babc5cbc4e5,Walt Disney World® Resort,DESTINATION,America/New_York,"{'id': 'b2148542-8777-4bdc-b2c3-f366d80281a1',..."


In [17]:
# expand 'children' column
rides_get_df['children'][0]

{'id': '8b5b9a00-b2cc-4bd9-bf39-9c312c17e8cb',
 'name': 'Sanaa Lounge',
 'entityType': 'RESTAURANT',
 'slug': 'sanaalounge',
 'externalId': '15602781;entityType=restaurant'}

In [18]:
# get ids for rides/attractions only
attractions = [ride['id'] for ride in rides_get_df['children'] if ride['entityType'] == 'ATTRACTION']

In [19]:
# api call to get coordinates for each ride id
coordinates = {}
for ride in attractions:
    ride_get = requests.get(f'https://api.themeparks.wiki/v1/entity/{ride}').json()
    coordinates[ride_get['name']] = (ride_get['location']['latitude'], ride_get['location']['longitude'])
    
with open('coordinates', 'wb') as f:
    dill.dump(coordinates, f)   

In [123]:
# create coordinate dictionary for each park
# ended up replacing api coordinates with gmaps coordinates for better accuracy
MK_coord = {'Space Mountain': (28.419201980778723, -81.5772043301858),
            "Buzz Lightyear": (28.418140529659667, -81.57970800504081),
            'Big Thunder Mtn': (28.420363191559492, -81.58479847483135),
            'Splash Mountain': (28.419301531937133, -81.58483619793391),
            "Peter Pan's Flight": (28.42031822118516, -81.58195711244385),
            'Winnie the Pooh': (28.420081267315698, -81.58091991300176),
            'Jungle Cruise': (28.4180104581068, -81.583468208738),
            '7 Dwarfs Train': (28.42068561007527, -81.57966639783211),
            'Under the Sea': (28.421054629551012, -81.58016876122969),
            'Magic Carpets': (28.418520878943376, -81.58367713666986),
            'Pirates of Caribbean': (28.418056396082246, -81.58424298326784),
            'Princess Hall: Cinderella': (28.42014556601841, -81.58015476000227),
            'Princess Hall: Tiana': (28.42014556601841, -81.58015476000227),
            'Haunted Mansion': (28.420137257117283, -81.58294497299853),
            'Regal Carrousel': (28.420274155776088, -81.58103942108878),
            'Dumbo': (28.42040455107225, -81.57903004710712),
            "it's a small world": (28.420621915256056, -81.58199483554829),
            'Mad Tea Party': (28.419957700391624, -81.57980906752827),
            "PhilharMagic": (28.4200425989214, -81.58159148850842),
            'Barnstormer': (28.420853489550115, -81.57867618550581),
            'Astro Orbiter': (28.41845452525681, -81.57924558600901),
            'Laugh Floor': (28.41841900537521, -81.57986822442584),
            "Tom'land Speedway": (28.419421976134636, -81.57911956586078),
            'PeopleMover': (28.41841900538073, -81.57886711120706),
            'Town Sq Mickey':(28.416861853098062, -81.57990799678127)}

HS_coord = {'Slinky Dog': (28.356566838524667, -81.56329942563616),
            'Alien Saucers': (28.3553504092135, -81.5629056353794),
            "Rock Coaster": (28.359618148460488, -81.56072991481986),
            'Toy Story Mania!': (28.356505923138783, -81.56196557155904)}

EC_coord = {'Spaceship Earth': (28.37542086382473, -81.54946777685959), 
             "Soarin'": (28.373986005531087, -81.55267569873278)}

AK_coord = {"It's Tough to Be a Bug": (28.357603297224315, -81.5899441481489),
            'Kilimanjaro Safaris': (28.36295471365359, -81.59382240288605),
            'Expedition Everest': (28.358429781123455, -81.5857526610227),
            'DINOSAUR': (28.35556946297818, -81.58889815320649),
            "Na'vi River": (28.35469077966241, -81.59184550190014),
            'Flight of Passage': (28.35480628295068, -81.59257979335898)}

MK_entrance = (28.41619616643251, -81.58119853273976)
AK_entrance = (28.35518331566209, -81.5900815235851)
EC_entrance = (28.37625808732175, -81.54944454986719)
HS_entrance = (28.35826216046488, -81.55887585232853)

#### Query google maps distance matrix for each ride origin/destination pair

In [145]:
# generate origin/destination pairs for all rides in each park
MK_paths = [path for path in combinations(MK_coord.items(), 2)]
HS_paths = [path for path in combinations(HS_coord.items(), 2)]
EC_paths = [path for path in combinations(EC_coord.items(), 2)]
AK_paths = [path for path in combinations(AK_coord.items(), 2)]

In [126]:
# requires Google Maps API key
API_key = ******* #enter Google Maps API key
gmaps = googlemaps.Client(key=API_key)

In [136]:
# function to query gmaps distance matrix for each origin/destination pair
# returns dataframe of travel durations in seconds and distances in meters
def from_entrance(entrance, coord_dict, gmaps_key):
    travel_df = pd.DataFrame(columns=['origin','destination','duration','distance'])
    n=0
    for key, value in coord_dict.items():
        o_coord = entrance
        d_coord = value 
        result = gmaps_key.distance_matrix(o_coord, d_coord, mode='walking')
        origin = 'entrance'
        destination = key
        result_dict = {'origin': origin,
                       'destination': destination,
                       'duration' : result['rows'][0]['elements'][0]['duration']['value'],
                       'distance': result['rows'][0]['elements'][0]['distance']['value']}
        travel_df.loc[n] = result_dict
        n+=1
    return travel_df

In [142]:
# function to query gmaps distance matrix for each origin/destination pair
# returns dataframe of travel durations in seconds and distances in meters
def get_travel(paths, gmaps_key):
    travel_df = pd.DataFrame(columns=['origin','destination','duration','distance'])
    for n in range(len(paths)):
        o_coord = paths[n][0][1]
        d_coord = paths[n][1][1]  
        result = gmaps_key.distance_matrix(o_coord, d_coord, mode='walking')
        origin = paths[n][0][0]
        destination = paths[n][1][0]
        result_dict = {'origin': origin,
                       'destination': destination,
                       'duration' : result['rows'][0]['elements'][0]['duration']['value'],
                       'distance': result['rows'][0]['elements'][0]['distance']['value']}
        travel_df.loc[n] = result_dict
    return travel_df

In [146]:
# make api calls for travel between rides
EC_travel = get_travel(EC_paths, gmaps)
AK_travel = get_travel(AK_paths, gmaps)
HS_travel = get_travel(HS_paths, gmaps)
MK_travel = get_travel(MK_paths, gmaps)

In [139]:
# make api calls for travel from entrance to first ride
EC_start = from_entrance(EC_entrance, EC_coord, gmaps)
MK_start = from_entrance(MK_entrance, MK_coord, gmaps)
AK_start = from_entrance(AK_entrance, AK_coord, gmaps)
HS_start = from_entrance(HS_entrance, HS_coord, gmaps)

In [147]:
# combine all travel info into one dataframe

travel_df = pd.concat([AK_start,
                       AK_travel,
                       EC_start,
                       EC_travel, 
                       HS_start,
                       HS_travel, 
                       MK_start,
                       MK_travel], axis=0, ignore_index=True)
travel_df.to_csv('travel_df.csv', index=False)